In [1]:
import numpy as np
from itertools import cycle

In [2]:
with open("ex.txt") as f:
    inp = f.read().strip()
# inp

In [3]:
a = cycle(inp)

# Part 1

In [4]:
class Chamber:
    def __init__(self, jets):
        self.rocks = cycle([
            np.array([[1,1,1,1]], bool),
            np.array([[0,1,0],
                      [1,1,1],
                      [0,1,0]], bool),
            np.array([[1,1,1],
                      [0,0,1],
                      [0,0,1]], bool),
            np.array([[1],[1],[1],[1]], bool),
            np.array([[1,1],
                      [1,1]], bool)
        ])

        self.jets = cycle(jets)
        
        max_height = 2022 * 13 // 5
        self.chamber = np.zeros((max_height,7), bool)
        
        self.top = 0
        
        self.rock_i = 0
        self.jet_i = 0

        
    def check_collision(self, rock, x, y):
        rh, rw = rock.shape
        return (self.chamber[y:y+rh, x:x+rw] & rock).any()
        
    def settle(self, rock, x, y):
        rh, rw = rock.shape
        self.chamber[y:y+rh,x:x+rw] |= rock
        
        # Find new highest point
        while self.chamber[self.top].any():
            self.top += 1
    
    def rockfall(self):
        rock = self.rocks.__next__()
        # print(rock)
        rh,rw = rock.shape
        x = 2
        y = self.top + 3
        
        while True:
            # Jet push
            jet = self.jets.__next__()
            # print(jet)
            if jet == '>':
                # print('>',end="")
                if x + rw < 7 and not self.check_collision(rock, x+1, y):
                    x += 1
            else:
                # print('<',end="")
                if x > 0 and not self.check_collision(rock, x-1, y):
                    x -= 1
            
            # Fall
            if not self.check_collision(rock, x, y-1) and y > 0:
                y -= 1
            else:
                self.settle(rock, x, y)
                return

        
        

In [5]:
rock = 1
chamber = Chamber(inp)
for i in range(2022):
    chamber.rockfall()
    # print(chamber)
    # if i == 5:
    #     break
chamber.top

3068

# Part 2

In [8]:
class Chamber:
    def __init__(self, jets):
        self.rocks = [
            np.array([[1,1,1,1]], bool),
            np.array([[0,1,0],
                      [1,1,1],
                      [0,1,0]], bool),
            np.array([[1,1,1],
                      [0,0,1],
                      [0,0,1]], bool),
            np.array([[1],[1],[1],[1]], bool),
            np.array([[1,1],
                      [1,1]], bool)
        ]

        self.jets = jets
        
        max_height = 10000
        self.chamber = np.zeros((max_height,7), bool)
        
        self.top = 0
        
        self.jet_i = 0
        self.rock_i = 0
        self.counter = 0
        
    def check_collision(self, rock, x, y):
        rh, rw = rock.shape
        return (self.chamber[y:y+rh, x:x+rw] & rock).any()
        
    def settle(self, rock, x, y):
        rh, rw = rock.shape
        self.counter += 1
        self.chamber[y:y+rh,x:x+rw] |= rock
        
        # Find new highest point
        while self.chamber[self.top].any():
            self.top += 1
            
    
    def rockfall(self):
        rock = self.rocks[self.rock_i]
        if self.rock_i < 4:
            self.rock_i += 1
        else: 
            self.rock_i = 0
        # print(rock)
        rh,rw = rock.shape
        x = 2
        y = self.top + 3
        
        while True:
            # Jet push
            jet = self.jets[self.jet_i]
            if self.jet_i < len(self.jets)-1:
                self.jet_i += 1
            else:
                self.jet_i = 0
            # print(jet)
            if jet == '>':
                # print('>',end="")
                if x + rw < 7 and not self.check_collision(rock, x+1, y):
                    x += 1
            else:
                # print('<',end="")
                if x > 0 and not self.check_collision(rock, x-1, y):
                    x -= 1
            
            # Fall
            if not self.check_collision(rock, x, y-1) and y > 0:
                y -= 1
            else:
                self.settle(rock, x, y)
                return
            
    def get_hash(self):
        return (self.jet_i * 6) + self.rock_i

        
        

In [9]:
with open("inp.txt") as f:
    inp = f.read().strip()

max_height=0
chamber = Chamber(inp)
seen = set()
cycle_seen = False
hash_val = 0
i = 0
while i < 1000000000000:
    if i == 3303:
        x = 1
    chamber.rockfall()
    if cycle_seen == False:
        if chamber.get_hash() in seen:
            cycle_seen = True
            height = chamber.top
            counter = chamber.counter
            hash_val = chamber.get_hash()
    else:
        if chamber.get_hash() == hash_val:
            cycle_height = chamber.top - height
            cycle_count = chamber.counter - counter
            times = (1000000000000-i)//cycle_count
            i += cycle_count * times
            max_height += cycle_height * times
            
    seen.add(chamber.get_hash())
    i+=1
    
max_height+=chamber.top
max_height

1602881844347